In [ ]:
# helpful munging scripts
# Callin Switzer
# 15 Jan 2016


In [2]:
# import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
 #%qtconsole

In [4]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [ ]:
aa = file_choose()

In [ ]:
print aa

In [ ]:
os.chdir('/Volumes/GMD/BeeSonicationLearningWithAvery/BeeFrequencyLearning/')

In [ ]:
fldrs = [f for f in os.listdir(".") if not f.startswith('.')]

In [ ]:
fldrs

In [ ]:
fldrs.sort(key=lambda x: os.path.getmtime(x)) # sort by time created

In [ ]:
# fldrs.reverse()
fldrs

In [ ]:
import csv

In [ ]:
with open("/Users/callinswitzer/Desktop/output.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(np.transpose(np.array([fldrs])))

In [ ]:
np.transpose(np.array([fldrs])).shape

In [ ]:
file_choose()

In [ ]:
# copy files and exclude movies
import shutil

In [ ]:
os.chdir('/Volumes/GMD/19 Dec Bee Sonication Backup/BeeSonicationLearningWithAvery/BeeFrequencyLearning/')

In [ ]:
sourceDir = os.getcwd()
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_NoImages'
IGNORE_PATTERNS = ('*.pgm')


In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
sourceDir = '/Volumes/GMD/BeeSonicationLearningWithAvery/HeavyLightFlowers/'
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_HeavyLight'
IGNORE_PATTERNS = ('*.pgm')


In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
# pick up here

In [ ]:
sourceDir = '/Volumes/GMD/BeeSonicationLearningWithAvery/TestAndTrainFakeAndRealFlowers/'
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_TestTrainDataset_noVid'
IGNORE_PATTERNS = ('*.pgm', '*.mp4')

In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
# done

In [26]:
os.chdir('/Users/callinswitzer/Desktop/SonicationLearningWAvery_NoImages/')

In [27]:
mypath = os.getcwd()

In [28]:
# get folder list
folders = [f for f in os.listdir(mypath) if "." not in f]

In [34]:
# for each folder, open it, and read the ampFreq.txt file
for ii in range(len(folders)):

    # read contents of each folder
    fcontents = [f for f in os.listdir(folders[ii]) if not f.startswith('.')]

    # get ampFreq file(s). there should be one per folder
    ampFreqFile = [x for x in fcontents if 'ampFreq.txt' in x]

    if len(ampFreqFile) > 1: 
        print("PROBLEM " + str(ii))


    # read ampFreq file
    tmpDF = pd.read_table(os.path.join(folders[ii], ampFreqFile[0]), header = None, sep = ',')

    # add folder name to data frame
    tmpDF[7] = folders[ii]

    # add frequency information
    folderInFolder = [x for x in fcontents if not 'ampFreq.txt' in x]

    if len(folderInFolder) > 1: 
        print("PROBLEM")

    # list files in inner folder
    accRecFiles = [x for x in os.listdir(os.path.join(folders[ii], folderInFolder[0])) if not x.startswith('.')]

    tmpDF[8] = accRecFiles

    # remove wingbeats
    tmpDF2 = tmpDF.ix[(tmpDF[0] > 220) & (tmpDF[0] < 450)]

    # renumber index
    tmpDF2.index = (np.arange(1, tmpDF2.shape[0] + 1))

    # write to .csv
    tmpDF.to_csv('/Users/callinswitzer/Desktop/FreqTrainCSVs/' + folders[ii] + '.csv', 
                 header = False, index = True)
    
    print ii

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231


In [35]:
## combine csv's all into a single file
csvDir = '/Users/callinswitzer/Desktop/FreqTrainCSVs/'

csvFiles = [f for f in os.listdir(csvDir) if f.endswith('.csv')]

In [40]:
frame = pd.DataFrame()
list_ = []
for file_ in csvFiles:
    df = pd.read_csv(csvDir + file_,index_col=None, header=0)
    list_.append(df)
    print file_
    
frame = pd.concat(list_)

BeeBlue2_28Nov2016_Hive4_high.csv
BeeBlue3_28Nov2016_Hive4_high.csv
BeeBlue4_28Nov2016_Hive4_high.csv
BeeBlue5_28Nov2016_Hive4_high.csv
BeeBlue6_28Nov2016_Hive4_high.csv
BeeBlue7_28Nov2016_Hive4_high.csv
Beeblue8_29Nov2016_Hive4_high.csv
BeeBlue_22Nov2016_Hive4_initial.csv
Beegold1_29Nov2016_Hive3_initial.csv
Beegoldred_02Dec2016_Hive4_initial.csv
Beegreen1_29Nov2016_Hive4_initial.csv
Beegreen2_02Dec2016_Hive4_initial.csv
Beelime1_12Dec2016_Hive5_initial.csv
Beelimeblue10_16Dec2016_Hive5_initial.csv
Beelimeblue1_14Dec2016_Hive5_initial.csv
Beelimeblue2_14Dec2016_Hive5_initial.csv
Beelimeblue3_14Dec2016_Hive5_initial.csv
Beelimeblue4_15Dec2016_Hive5_initial.csv
Beelimeblue5_15Dec2016_Hive5_initial.csv
Beelimeblue6_15Dec2016_Hive5_initial.csv
Beelimeblue7_15Dec2016_Hive5_initial.csv
Beelimeblue8_15Dec2016_Hive5_initial.csv
Beelimeblue9_15Dec2016_Hive5_initial.csv
Beelimegold1_18Dec2016_Hive5_initial.csv
Beelimegold1_18Dec2016_Hive5_NoReward.csv
Beelimegreen1_18Dec2016_Hive5_initial.csv
B

In [41]:
frame.to_csv('/Users/callinswitzer/Desktop/freqLearn.csv', header = False, index = False)